# Run model module on GCP

In [ ]:
import datetime
import json
import math
import os
import sys
import tensorflow as tf

import os
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from proganomaly_modules.training_module.trainer import gan_layer_architecture_shapes
from proganomaly_modules.training_module.trainer import image_masks

In [ ]:
PROJECT = "..."
BUCKET = "..."
REGION = "us-central1"

# Import os environment variables
os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] =  BUCKET
os.environ["REGION"] = REGION
os.environ["TFVERSION"] = "2.3"
os.environ["PYTHON_VERSION"] = "3.7"

These parameter configs are used to create JSON. The idea is to use the JSON as a baseline and then only override a small number of values via the command line.

In [ ]:
# Generator hyperparameters.
generator_dict = dict()
# Which paper to use for generator architecture: "berg", "GANomaly".
generator_dict["architecture"] = "GANomaly"

# Whether generator will be trained or not.
generator_dict["train"] = True
# Number of steps to train generator for per cycle.
generator_dict["train_steps"] = 1

# The latent size of the berg input noise vector or the GANomaly generator's
# encoder logits vector.
generator_dict["latent_size"] = 512
# Whether to normalize latent vector before projection.
generator_dict["normalize_latents"] = True
# Whether to use pixel norm op after each convolution.
generator_dict["use_pixel_norm"] = True
# Small value to add to denominator for numerical stability.
generator_dict["pixel_norm_epsilon"] = 1e-8
# The 3D dimensions to project latent noise vector into.
generator_dict["projection_dims"] = [4, 4, 512]
# The amount of leakyness of generator's leaky relus.
generator_dict["leaky_relu_alpha"] = 0.2
# The final activation function of generator: None, sigmoid, tanh, relu.
generator_dict["final_activation"] = "None"
# Whether to add uniform noise to fake images.
generator_dict["add_uniform_noise_to_fake_images"] = True
# Scale factor for L1 regularization for generator.
generator_dict["l1_regularization_scale"] = 0.
# Scale factor for L2 regularization for generator.
generator_dict["l2_regularization_scale"] = 0.
# Name of optimizer to use for generator.
generator_dict["optimizer"] = "Adam"
# How quickly we train model by scaling the gradient for generator.
generator_dict["learning_rate"] = 0.001
# Adam optimizer's beta1 hyperparameter for first moment.
generator_dict["adam_beta1"] = 0.0
# Adam optimizer's beta2 hyperparameter for second moment.
generator_dict["adam_beta2"] = 0.99
# Adam optimizer's epsilon hyperparameter for numerical stability.
generator_dict["adam_epsilon"] = 1e-8
# Global clipping to prevent gradient norm to exceed this value for generator.
generator_dict["clip_gradients"] = None

generator_berg_dict = dict()
generator_ganomaly_dict = dict()

generator_berg_losses_dict = dict()
generator_ganomaly_losses_dict = dict()
if generator_dict["architecture"] == "berg":
    # The latent vector's random normal mean.
    generator_berg_dict["latent_mean"] = 0.0
    # The latent vector's random normal standard deviation.
    generator_berg_dict["latent_stddev"] = 1.0

    # These are just example values, yours will vary.
    # Weights to multiply loss of D(G(z))
    generator_berg_losses_dict["D_of_G_of_z_loss_weight"] = 1.0

    # Weights to multiply loss of D(G(E(x)))
    generator_berg_losses_dict["D_of_G_of_E_of_x_loss_weight"] = 0.0
    # Weights to multiply loss of D(G(E(G(z)))
    generator_berg_losses_dict["D_of_G_of_E_of_G_of_z_loss_weight"] = 0.0

    # Weights to multiply loss of z - E(G(z))
    generator_berg_losses_dict["z_minus_E_of_G_of_z_l1_loss_weight"] = 0.0
    generator_berg_losses_dict["z_minus_E_of_G_of_z_l2_loss_weight"] = 0.0
    # Weights to multiply loss of G(z) - G(E(G(z))
    generator_berg_losses_dict["G_of_z_minus_G_of_E_of_G_of_z_l1_loss_weight"] = 0.0
    generator_berg_losses_dict["G_of_z_minus_G_of_E_of_G_of_z_l2_loss_weight"] = 0.0
    # Weights to multiply loss of E(x) - E(G(E(x)))
    generator_berg_losses_dict["E_of_x_minus_E_of_G_of_E_of_x_l1_loss_weight"] = 1.0
    generator_berg_losses_dict["E_of_x_minus_E_of_G_of_E_of_x_l2_loss_weight"] = 0.0
    # Weights to multiply loss of x - G(E(x))
    generator_berg_losses_dict["x_minus_G_of_E_of_x_l1_loss_weight"] = 0.0
    generator_berg_losses_dict["x_minus_G_of_E_of_x_l2_loss_weight"] = 0.0

    # GANomaly parameters to zero.
    # Weights to multiply loss of D(G(x))
    generator_ganomaly_losses_dict["D_of_G_of_x_loss_weight"] = 0.0

    # Weights to multiply loss of x - G(x)
    generator_ganomaly_losses_dict["x_minus_G_of_x_l1_loss_weight"] = 0.0
    generator_ganomaly_losses_dict["x_minus_G_of_x_l2_loss_weight"] = 0.0

    # Weights to multiply loss of Ge(x) - E(G(x))
    generator_ganomaly_losses_dict["Ge_of_x_minus_E_of_G_of_x_l1_loss_weight"] = 0.0
    generator_ganomaly_losses_dict["Ge_of_x_minus_E_of_G_of_x_l2_loss_weight"] = 0.0
else:  # GANomaly
    # Whether generator GANomaly architecture uses U-net skip connection for each block.
    generator_ganomaly_dict["use_unet_skip_connections"] = [True] * 9

    # Percent of masking image inputs to generator.
    generator_ganomaly_dict["mask_generator_input_images_percent"] = 0.2
    # Integer amount to randomly shift image mask block sizes.
    generator_ganomaly_dict["image_mask_block_random_shift_amount"] = 0
    # Whether to use shuffle or dead image block masking.
    generator_ganomaly_dict["use_shuffle_image_masks"] = True
    # Whether to add uniform noise to GANomaly Z vector.
    generator_ganomaly_dict["add_uniform_noise_to_z"] = True

    # These are just example values, yours will vary.
    # Weights to multiply loss of D(G(x))
    generator_ganomaly_losses_dict["D_of_G_of_x_loss_weight"] = 1.0

    # Weights to multiply loss of x - G(x)
    generator_ganomaly_losses_dict["x_minus_G_of_x_l1_loss_weight"] = 0.0
    generator_ganomaly_losses_dict["x_minus_G_of_x_l2_loss_weight"] = 1.0

    # Weights to multiply loss of Ge(x) - E(G(x))
    generator_ganomaly_losses_dict["Ge_of_x_minus_E_of_G_of_x_l1_loss_weight"] = 0.0
    generator_ganomaly_losses_dict["Ge_of_x_minus_E_of_G_of_x_l2_loss_weight"] = 0.0

    # Berg parameters to zero.
    # Weights to multiply loss of D(G(z))
    generator_berg_losses_dict["D_of_G_of_z_loss_weight"] = 0.0

    # Weights to multiply loss of D(G(E(x)))
    generator_berg_losses_dict["D_of_G_of_E_of_x_loss_weight"] = 0.0
    # Weights to multiply loss of D(G(E(G(z)))
    generator_berg_losses_dict["D_of_G_of_E_of_G_of_z_loss_weight"] = 0.0

    # Weights to multiply loss of z - E(G(z))
    generator_berg_losses_dict["z_minus_E_of_G_of_z_l1_loss_weight"] = 0.0
    generator_berg_losses_dict["z_minus_E_of_G_of_z_l2_loss_weight"] = 0.0
    # Weights to multiply loss of G(z) - G(E(G(z))
    generator_berg_losses_dict["G_of_z_minus_G_of_E_of_G_of_z_l1_loss_weight"] = 0.0
    generator_berg_losses_dict["G_of_z_minus_G_of_E_of_G_of_z_l2_loss_weight"] = 0.0
    # Weights to multiply loss of E(x) - E(G(E(x)))
    generator_berg_losses_dict["E_of_x_minus_E_of_G_of_E_of_x_l1_loss_weight"] = 0.0
    generator_berg_losses_dict["E_of_x_minus_E_of_G_of_E_of_x_l2_loss_weight"] = 0.0
    # Weights to multiply loss of x - G(E(x))
    generator_berg_losses_dict["x_minus_G_of_E_of_x_l1_loss_weight"] = 0.0
    generator_berg_losses_dict["x_minus_G_of_E_of_x_l2_loss_weight"] = 0.0

generator_dict["berg"] = generator_berg_dict
generator_dict["GANomaly"] = generator_ganomaly_dict

generator_dict["losses"] = {}
generator_dict["losses"]["berg"] = generator_berg_losses_dict
generator_dict["losses"]["GANomaly"] = generator_ganomaly_losses_dict

generator_dict

In [ ]:
# Encoder hyperparameters.
encoder_dict = dict()
# These are optional if using GANomaly architecture, required for berg.
# Whether encoder will be created or not.
encoder_dict["create"] = False
# Whether encoder will be trained or not.
encoder_dict["train"] = False

# Whether to use minibatch stddev op before first base conv layer.
encoder_dict["use_minibatch_stddev"] = True
# The size of groups to split minibatch examples into.
encoder_dict["minibatch_stddev_group_size"] = 4
# Whether to average across feature maps and pixels for minibatch stddev.
encoder_dict["minibatch_stddev_use_averaging"] = True
# The amount of leakyness of encoder's leaky relus.
encoder_dict["leaky_relu_alpha"] = 0.2
# Scale factor for L1 regularization for encoder.
encoder_dict["l1_regularization_scale"] = 0.
# Scale factor for L2 regularization for encoder.
encoder_dict["l2_regularization_scale"] = 0.
# Name of optimizer to use for encoder.
encoder_dict["optimizer"] = "Adam"
# How quickly we train model by scaling the gradient for encoder.
encoder_dict["learning_rate"] = 0.001
# Adam optimizer's beta1 hyperparameter for first moment.
encoder_dict["adam_beta1"] = 0.0
# Adam optimizer's beta2 hyperparameter for second moment.
encoder_dict["adam_beta2"] = 0.99
# Adam optimizer's epsilon hyperparameter for numerical stability.
encoder_dict["adam_epsilon"] = 1e-8
# Global clipping to prevent gradient norm to exceed this value for encoder.
encoder_dict["clip_gradients"] = None

encoder_losses_dict = dict()
# Berg Losses
encoder_losses_berg_dict = dict()
# Weights to multiply loss of D(G(E(x)))
encoder_losses_berg_dict["D_of_G_of_E_of_x_loss_weight"] = 0.0
# Weights to multiply loss of D(G(E(G(z)))
encoder_losses_berg_dict["D_of_G_of_E_of_G_of_z_loss_weight"] = 0.0

# Weights to multiply loss of z - E(G(z))
encoder_losses_berg_dict["z_minus_E_of_G_of_z_l1_loss_weight"] = 0.0
encoder_losses_berg_dict["z_minus_E_of_G_of_z_l2_loss_weight"] = 0.0
# Weights to multiply loss of G(z) - G(E(G(z))
encoder_losses_berg_dict["G_of_z_minus_G_of_E_of_G_of_z_l1_loss_weight"] = 0.0
encoder_losses_berg_dict["G_of_z_minus_G_of_E_of_G_of_z_l2_loss_weight"] = 0.0
# Weights to multiply loss of E(x) - E(G(E(x)))
encoder_losses_berg_dict["E_of_x_minus_E_of_G_of_E_of_x_l1_loss_weight"] = 0.0
encoder_losses_berg_dict["E_of_x_minus_E_of_G_of_E_of_x_l2_loss_weight"] = 0.0
# Weights to multiply loss of x - G(E(x))
encoder_losses_berg_dict["x_minus_G_of_E_of_x_l1_loss_weight"] = 0.0
encoder_losses_berg_dict["x_minus_G_of_E_of_x_l2_loss_weight"] = 0.0

# GANomaly Losses
encoder_losses_ganomaly_dict = dict()
# Weights to multiply loss of Ge(x) - E(G(x))
encoder_losses_ganomaly_dict["Ge_of_x_minus_E_of_G_of_x_l1_loss_weight"] = 0.0
encoder_losses_ganomaly_dict["Ge_of_x_minus_E_of_G_of_x_l2_loss_weight"] = 1.0

encoder_losses_dict["berg"] = encoder_losses_berg_dict
encoder_losses_dict["GANomaly"] = encoder_losses_ganomaly_dict
encoder_dict["losses"] = encoder_losses_dict

encoder_dict

In [ ]:
# Discriminator hyperparameters.
discriminator_dict = dict()
# Whether discriminator will be created or not.
discriminator_dict["create"] = True
# Whether discriminator will be trained or not.
discriminator_dict["train"] = True
# Number of steps to train discriminator for per cycle.
discriminator_dict["train_steps"] = 1

# Whether to use minibatch stddev op before first base conv layer.
discriminator_dict["use_minibatch_stddev"] = True
# The size of groups to split minibatch examples into.
discriminator_dict["minibatch_stddev_group_size"] = 4
# Whether to average across feature maps and pixels for minibatch stddev.
discriminator_dict["minibatch_stddev_use_averaging"] = True
# The amount of leakyness of discriminator's leaky relus.
discriminator_dict["leaky_relu_alpha"] = 0.2
# Scale factor for L1 regularization for discriminator.
discriminator_dict["l1_regularization_scale"] = 0.
# Scale factor for L2 regularization for discriminator.
discriminator_dict["l2_regularization_scale"] = 0.
# Name of optimizer to use for discriminator.
discriminator_dict["optimizer"] = "Adam"
# How quickly we train model by scaling the gradient for discriminator.
discriminator_dict["learning_rate"] = 0.001
# Adam optimizer's beta1 hyperparameter for first moment.
discriminator_dict["adam_beta1"] = 0.0
# Adam optimizer's beta2 hyperparameter for second moment.
discriminator_dict["adam_beta2"] = 0.99
# Adam optimizer's epsilon hyperparameter for numerical stability.
discriminator_dict["adam_epsilon"] = 1e-8
# Global clipping to prevent gradient norm to exceed this value for discriminator.
discriminator_dict["clip_gradients"] = None
# Coefficient of gradient penalty for discriminator.
discriminator_dict["gradient_penalty_coefficient"] = 10.0
# Target value of gradient magnitudes for gradient penalty for discriminator.
discriminator_dict["gradient_penalty_target"] = 1.0
# Coefficient of epsilon drift penalty for discriminator.
discriminator_dict["epsilon_drift"] = 0.001

# Losses
discriminator_losses_dict = dict()
# Weight to multiply loss of D(x)
discriminator_losses_dict["D_of_x_loss_weight"] = 1.0

# Berg Losses
discriminator_losses_berg_dict = dict()
# Weight to multiply loss of D(G(z))
discriminator_losses_berg_dict["D_of_G_of_z_loss_weight"] = 0.0
# Weight to multiply loss of D(G(E(x)))
discriminator_losses_berg_dict["D_of_G_of_E_of_x_loss_weight"] = 0.0
# Weight to multiply loss of D(G(E(G(z)))
discriminator_losses_berg_dict["D_of_G_of_E_of_G_of_z_loss_weight"] = 0.0

# GANomaly Losses
discriminator_losses_ganomaly_dict = dict()
# Weight to multiply loss of D(G(x))
discriminator_losses_ganomaly_dict["D_of_G_of_x_loss_weight"] = 1.0

discriminator_losses_dict["berg"] = discriminator_losses_berg_dict
discriminator_losses_dict["GANomaly"] = discriminator_losses_ganomaly_dict
discriminator_dict["losses"] = discriminator_losses_dict

discriminator_dict

In [ ]:
# Reconstruction training parameters.
reconstruction_dict = dict()
# Whether using multiple resolutions across a list of TF Records.
reconstruction_dict["use_multiple_resolution_records"] = True
# GCS locations to read reconstruction training data.
reconstruction_dict["train_file_patterns"] = [
    tf.io.gfile.glob(
        pattern="gs://.../data/*/*.svs.{}.*.tfrecords".format(i)
    )[0:150]
    for i in range(9 - 1, -1, -1)
]
# GCS locations to read reconstruction evaluation data.
reconstruction_dict["eval_file_patterns"] = [
    tf.io.gfile.glob(
        pattern="gs://.../data/*/*.svs.{}.*.tfrecords".format(i)
    )[0:150]
    for i in range(9 - 1, -1, -1)
]
# Which dataset to use for reconstruction training:
# "mnist", "cifar10", "cifar10_car", "tf_record"
reconstruction_dict["dataset"] = "tf_record"
# TF Record Example feature schema for reconstruction.
reconstruction_dict["tf_record_example_schema"] = [
    {
        "name": "image/encoded",
        "type": "FixedLen",
        "shape": [],
        "dtype": "str"
    },
    {
        "name": "image/name",
        "type": "FixedLen",
        "shape": [],
        "dtype": "str"
    }
]
# Name of image feature within schema dictionary.
reconstruction_dict["image_feature_name"] = "image/encoded"
# Encoding of image: raw, png, or jpeg.
reconstruction_dict["image_encoding"] = "png"
# Height of predownscaled image if NOT using multiple resolution records.
reconstruction_dict["image_predownscaled_height"] = 1024
# Width of predownscaled image if NOT using multiple resolution records.
reconstruction_dict["image_predownscaled_width"] = 1024
# Depth of image, number of channels.
reconstruction_dict["image_depth"] = 3
# Name of label feature within schema dictionary.
reconstruction_dict["label_feature_name"] = ""
# Schedule list of number of epochs to train for reconstruction.
reconstruction_dict["num_epochs_schedule"] = [1] * 9
# Number of examples in one epoch of reconstruction training set.
reconstruction_dict["train_dataset_length"] = 330415
# Schedule list of number of examples in reconstruction training batch for each resolution block.
reconstruction_dict["train_batch_size_schedule"] = [32] + [16] * 4 + [4] + [2] * 2 + [1]
# Schedule list of number of examples in reconstruction evaluation batch for each resolution block.
reconstruction_dict["eval_batch_size_schedule"] = [32] + [16] * 4 + [4] + [2] * 2 + [1]
# Number of steps/batches to evaluate for reconstruction.
reconstruction_dict["eval_steps"] = 1
# List of number of examples until block added to networks.
reconstruction_dict["num_examples_until_growth_schedule"] = [
    epochs * reconstruction_dict["train_dataset_length"]
    for epochs in reconstruction_dict["num_epochs_schedule"]
]
# List of number of steps/batches until block added to networks.
reconstruction_dict["num_steps_until_growth_schedule"] = [
    ex // bs
    for ex, bs in zip(
        reconstruction_dict["num_examples_until_growth_schedule"],
        reconstruction_dict["train_batch_size_schedule"]
    )
]
# Whether to autotune input function performance for reconstruction datasets.
reconstruction_dict["input_fn_autotune"] = True
# How many steps to train before writing steps and loss to log.
reconstruction_dict["log_step_count_steps"] = 100
# How many steps to train before saving a summary.
reconstruction_dict["save_summary_steps"] = 100
# Whether to write loss summaries for TensorBoard.
reconstruction_dict["write_loss_summaries"] = False
# Whether to write generator image summaries for TensorBoard.
reconstruction_dict["write_generator_image_summaries"] = False
# Whether to write encoder image summaries for TensorBoard.
reconstruction_dict["write_encoder_image_summaries"] = False
# Whether to write variable histogram summaries for TensorBoard.
reconstruction_dict["write_variable_histogram_summaries"] = False
# Whether to write gradient histogram summaries for TensorBoard.
reconstruction_dict["write_gradient_histogram_summaries"] = False
# How many steps to train reconstruction before saving a checkpoint.
reconstruction_dict["save_checkpoints_steps"] = 10000
# Max number of reconstruction checkpoints to keep.
reconstruction_dict["keep_checkpoint_max"] = 100
# Whether to save checkpoint every growth phase.
reconstruction_dict["checkpoint_every_growth_phase"] = True
# Whether to save checkpoint every epoch.
reconstruction_dict["checkpoint_every_epoch"] = True
# Checkpoint growth index to restore checkpoint.
reconstruction_dict["checkpoint_growth_idx"] = 0
# Checkpoint epoch index to restore checkpoint.
reconstruction_dict["checkpoint_epoch_idx"] = 0
# The checkpoint save path for saving and restoring.
reconstruction_dict["checkpoint_save_path"] = ""
# Whether to store loss logs.
reconstruction_dict["store_loss_logs"] = True
# Whether to normalize loss logs.
reconstruction_dict["normalized_loss_logs"] = True
# Whether to print model summaries.
reconstruction_dict["print_training_model_summaries"] = False
# Initial growth index to resume training midway.
reconstruction_dict["initial_growth_idx"] = 0
# Initial epoch index to resume training midway.
reconstruction_dict["initial_epoch_idx"] = 0
# Max number of times training loop can be restarted such as for NaN losses.
reconstruction_dict["max_training_loop_restarts"] = 20

# Whether to scale layer weights to equalize learning rate each forward pass.
reconstruction_dict["use_equalized_learning_rate"] = True
# Whether to normalize reconstruction losses by number of pixels.
reconstruction_dict["normalize_reconstruction_losses"] = True

reconstruction_dict

In [ ]:
# Error distribution training parameters.
error_distribution_dict = dict()
# Whether using multiple resolutions across a list of TF Records.
error_distribution_dict["use_multiple_resolution_records"] = False
# GCS locations to read error distribution training data.
error_distribution_dict["train_file_pattern"] = tf.io.gfile.glob(
    pattern="gs://.../data/*/*.svs.{}.*.tfrecords".format(0)
)[150:175]
# GCS locations to read error distribution training data.
error_distribution_dict["eval_file_pattern"] = tf.io.gfile.glob(
    pattern="gs://.../data/*/*.svs.{}.*.tfrecords".format(0)
)[150:175]
# Which dataset to use for error distribution training:
# "mnist", "cifar10", "cifar10_car", "tf_record"
error_distribution_dict["dataset"] = "tf_record"
# TF Record Example feature schema for error distribution.
error_distribution_dict["tf_record_example_schema"] = [
    {
        "name": "image/encoded",
        "type": "FixedLen",
        "shape": [],
        "dtype": "str"
    },
    {
        "name": "image/name",
        "type": "FixedLen",
        "shape": [],
        "dtype": "str"
    }
]
# Name of image feature within schema dictionary.
error_distribution_dict["image_feature_name"] = "image/encoded"
# Encoding of image: raw, png, or jpeg.
error_distribution_dict["image_encoding"] = "png"
# Height of predownscaled image if NOT using multiple resolution records.
error_distribution_dict["image_predownscaled_height"] = 1024
# Width of predownscaled image if NOT using multiple resolution records.
error_distribution_dict["image_predownscaled_width"] = 1024
# Depth of image, number of channels.
error_distribution_dict["image_depth"] = 3
# Name of label feature within schema dictionary.
error_distribution_dict["label_feature_name"] = ""
# Number of examples in one epoch of error distribution training set.
error_distribution_dict["train_dataset_length"] = 44693
# Number of examples in error distribution training batch.
error_distribution_dict["train_batch_size"] = 16
# Number of steps/batches to evaluate for error distribution.
error_distribution_dict["eval_steps"] = 1
# Whether to autotune input function performance for error distribution datasets.
error_distribution_dict["input_fn_autotune"] = True
# How many steps to train error distribution before saving a checkpoint.
error_distribution_dict["save_checkpoints_steps"] = 10000
# Max number of error distribution checkpoints to keep.
error_distribution_dict["keep_checkpoint_max"] = 100
# The checkpoint save path for saving and restoring.
error_distribution_dict["checkpoint_save_path"] = ""
# Max number of times training loop can be restarted.
error_distribution_dict["max_training_loop_restarts"] = 20

# Whether using sample or population covariance for error distribution.
error_distribution_dict["use_sample_covariance"] = True

error_distribution_dict

In [ ]:
# Dynamic threshold training parameters.
dynamic_threshold_dict = dict()
# Whether using multiple resolutions across a list of TF Records.
dynamic_threshold_dict["use_multiple_resolution_records"] = False
# GCS locations to read dynamic threshold training data.
dynamic_threshold_dict["train_file_pattern"] = tf.io.gfile.glob(
    pattern="gs://.../data/*/*.svs.{}.*.tfrecords".format(0)
)[175:200]
# GCS locations to read dynamic threshold evaluation data.
dynamic_threshold_dict["eval_file_pattern"] = tf.io.gfile.glob(
    pattern="gs://.../data/*/*.svs.{}.*.tfrecords".format(0)
)[175:200]
# Which dataset to use for dynamic threshold training:
# "mnist", "cifar10", "cifar10_car", "tf_record"
dynamic_threshold_dict["dataset"] = "tf_record"
# TF Record Example feature schema for dynamic threshold.
dynamic_threshold_dict["tf_record_example_schema"] = [
    {
        "name": "image/encoded",
        "type": "FixedLen",
        "shape": [],
        "dtype": "str"
    },
    {
        "name": "image/name",
        "type": "FixedLen",
        "shape": [],
        "dtype": "str"
    }
]
# Name of image feature within schema dictionary.
dynamic_threshold_dict["image_feature_name"] = "image/encoded"
# Encoding of image: raw, png, or jpeg.
dynamic_threshold_dict["image_encoding"] = "png"
# Height of predownscaled image if NOT using multiple resolution records.
dynamic_threshold_dict["image_predownscaled_height"] = 1024
# Width of predownscaled image if NOT using multiple resolution records.
dynamic_threshold_dict["image_predownscaled_width"] = 1024
# Depth of image, number of channels.
dynamic_threshold_dict["image_depth"] = 3
# Name of label feature within schema dictionary.
dynamic_threshold_dict["label_feature_name"] = ""
# Number of examples in one epoch of dynamic threshold training set.
dynamic_threshold_dict["train_dataset_length"] = 52517
# Number of examples in dynamic threshold training batch.
dynamic_threshold_dict["train_batch_size"] = 16
# Number of steps/batches to evaluate for dynamic threshold.
dynamic_threshold_dict["eval_steps"] = 1
# Whether to autotune input function performance for dynamic threshold datasets.
dynamic_threshold_dict["input_fn_autotune"] = True
# How many steps to train dynamic threshold before saving a checkpoint.
dynamic_threshold_dict["save_checkpoints_steps"] = 10000
# Max number of dynamic threshold checkpoints to keep.
dynamic_threshold_dict["keep_checkpoint_max"] = 100
# The checkpoint save path for saving and restoring.
dynamic_threshold_dict["checkpoint_save_path"] = ""
# Max number of times training loop can be restarted.
dynamic_threshold_dict["max_training_loop_restarts"] = 20

# Whether using supervised dynamic thresholding or unsupervised.
dynamic_threshold_dict["use_supervised"] = False

supervised_dict = dict()
# Beta value for supervised F-beta score.
supervised_dict["f_score_beta"] = 0.05

unsupervised_dict = dict()
# Whether using sample or population covariance for dynamic threshold.
unsupervised_dict["use_sample_covariance"] = True
# Max standard deviations of Mahalanobis distance to flag as outlier.
unsupervised_dict["max_mahalanobis_stddevs"] = 3.0

dynamic_threshold_dict["supervised"] = supervised_dict
dynamic_threshold_dict["unsupervised"] = unsupervised_dict

dynamic_threshold_dict

In [ ]:
# Training parameters.
training_dict = dict()
# GCS location to write checkpoints, loss logs, and export models.
training_dict["output_dir"] = "gs://my-bucket/trained_models/experiment"
# Version of TensorFlow.
training_dict["tf_version"] = 2.3
# Whether to use graph mode or not (eager).
training_dict["use_graph_mode"] = True
# Which distribution strategy to use, if any.
training_dict["distribution_strategy"] = "Mirrored"
# Whether we subclass models or use Functional API.
training_dict["subclass_models"] = True
# Whether performing training phase 1 or not.
training_dict["train_reconstruction"] = True
# Whether performing training phase 2 or not.
training_dict["train_error_distribution"] = True
# Whether performing training phase 3 or not.
training_dict["train_dynamic_threshold"] = True

training_dict["reconstruction"] = reconstruction_dict
training_dict["error_distribution"] = error_distribution_dict
training_dict["dynamic_threshold"] = dynamic_threshold_dict

training_dict

In [ ]:
# Export parameters.
export_dict = dict()

# Most recent export's growth index so that there are no repeat exports.
export_dict["most_recent_export_growth_idx"] = -1
# Most recent export's epoch index so that there are no repeat exports.
export_dict["most_recent_export_epoch_idx"] = -1
# Whether to export SavedModel every growth phase.
export_dict["export_every_growth_phase"] = True
# Whether to export SavedModel every epoch.
export_dict["export_every_epoch"] = True
# Whether to export all growth phases or just current.
export_dict["export_all_growth_phases"] = True

# Using a random noise vector Z with shape (batch_size, generator_latent_size) for berg.
# Whether to export Z.
export_dict["export_Z"] = True
# Whether to export generated images, G(z).
export_dict["export_generated_images"] = True
# Whether to export encoded generated logits, E(G(z)).
export_dict["export_encoded_generated_logits"] = True
# Whether to export encoded generated images, G(E(G(z))).
export_dict["export_encoded_generated_images"] = True
# Whether to export Z generated images, Gd(z).
export_dict["export_Z_generated_images"] = True

# Using a query image with shape (batch_size, height, width, depth)
# Whether to export query images.
export_dict["export_query_images"] = True

# Berg encoded exports.
# Whether to export encoded query logits, E(x).
export_dict["export_query_encoded_logits"] = True
# Whether to export encoded query images, G(E(x)).
export_dict["export_query_encoded_images"] = True

# GANomaly encoded exports.
# Whether to export generator encoded query logits, Ge(x).
export_dict["export_query_gen_encoded_logits"] = True
# Whether to export generator encoded query images, G(x) = Gd(Ge(x)).
export_dict["export_query_gen_encoded_images"] = True
# Whether to export encoder encoded query logits, E(G(x)).
export_dict["export_query_enc_encoded_logits"] = True
# Whether to export encoder encoded query images, Gd(E(G(x))).
export_dict["export_query_enc_encoded_images"] = True

# Anomaly exports.
# Whether to export query anomaly images using sigmoid scaling.
export_dict["export_query_anomaly_images_sigmoid"] = True
# Whether to export query anomaly images using linear scaling.
export_dict["export_query_anomaly_images_linear"] = True
# Whether to export query Mahalanobis distances.
export_dict["export_query_mahalanobis_distances"] = True
# Whether to export query Mahalanobis distance images using sigmoid scaling.
export_dict["export_query_mahalanobis_distance_images_sigmoid"] = True
# Whether to export query Mahalanobis distance images using linear scaling.
export_dict["export_query_mahalanobis_distance_images_linear"] = True
# Whether to export query pixel anomaly flag binary images.
export_dict["export_query_pixel_anomaly_flag_images"] = True
# Whether to export query pixel anomaly flag binary images.
export_dict["export_query_pixel_anomaly_flag_counts"] = True
# Whether to export query pixel anomaly flag binary images.
export_dict["export_query_pixel_anomaly_flag_percentages"] = True
# Whether to export query anomaly scores, only for Berg.
export_dict["export_query_anomaly_scores"] = False
# Whether to export query anomaly flags, only for Berg.
export_dict["export_query_anomaly_flags"] = False

# Anomaly parameters.
# The threshold value at which above flags scores images as anomalous.
export_dict["anomaly_threshold"] = 5.0
# The anomaly convex combination factor for weighting the two anomaly losses.
export_dict["anom_convex_combo_factor"] = 0.05

# Whether to print model summaries.
export_dict["print_serving_model_summaries"] = False

export_dict

In [ ]:
# Full parameters.
arguments = dict()

arguments["generator"] = generator_dict
arguments["encoder"] = encoder_dict
arguments["discriminator"] = discriminator_dict
arguments["training"] = training_dict
arguments["export"] = export_dict

# Full lists for full 1024x1024 network growth.
full_conv_num_filters = [[512, 512], [512, 512], [512, 512], [512, 512], [256, 256], [128, 128], [64, 64], [32, 32], [16, 16]]
full_conv_kernel_sizes = [[4, 3], [3, 3], [3, 3], [3, 3], [3, 3], [3, 3], [3, 3], [3, 3], [3, 3]]
full_conv_strides = [[1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1]]

# Set final image size as a multiple of 2, starting at 4.
image_size = 1024
num_conv_blocks = max(
    min(int(math.log(image_size, 2) - 1), len(full_conv_num_filters)), 1
)

arguments["conv_num_filters"] = full_conv_num_filters[0:num_conv_blocks]
arguments["conv_kernel_sizes"] = full_conv_kernel_sizes[0:num_conv_blocks]
arguments["conv_strides"] = full_conv_strides[0:num_conv_blocks]

# Get conv layer properties for generator and discriminator.
(generator,
 discriminator) = (
    gan_layer_architecture_shapes.calc_generator_discriminator_conv_layer_properties(
        arguments["conv_num_filters"],
        arguments["conv_kernel_sizes"],
        arguments["conv_strides"],
        arguments["training"]["reconstruction"]["image_depth"]
    )
)

# Split up generator properties into separate lists.
(generator_base_conv_blocks,
 generator_growth_conv_blocks,
 generator_to_rgb_layers) = (
    gan_layer_architecture_shapes.split_up_generator_conv_layer_properties(
        generator,
        arguments["conv_num_filters"],
        arguments["conv_strides"],
        arguments["training"]["reconstruction"]["image_depth"]
    )
)

# Generator list of list of lists of base conv block layer shapes.
arguments["generator"]["base_conv_blocks"] = generator_base_conv_blocks
# Generator list of list of lists of growth conv block layer shapes.
arguments["generator"]["growth_conv_blocks"] = generator_growth_conv_blocks
# Generator list of list of lists of to_RGB layer shapes.
arguments["generator"]["to_rgb_layers"] = generator_to_rgb_layers

# Split up discriminator properties into separate lists.
(discriminator_from_rgb_layers,
 discriminator_base_conv_blocks,
 discriminator_growth_conv_blocks) = (
    gan_layer_architecture_shapes.split_up_discriminator_conv_layer_properties(
        discriminator,
        arguments["conv_num_filters"],
        arguments["conv_strides"],
        arguments["training"]["reconstruction"]["image_depth"]
    )
)

# Discriminator list of list of lists of from_RGB layer shapes.
arguments["discriminator"]["from_rgb_layers"] = discriminator_from_rgb_layers
# Discriminator list of list of lists of base conv block layer shapes.
arguments["discriminator"]["base_conv_blocks"] = (
    discriminator_base_conv_blocks
)
# Discriminator list of list of lists of growth conv block layer shapes.
arguments["discriminator"]["growth_conv_blocks"] = (
    discriminator_growth_conv_blocks
)

if (arguments["generator"]["architecture"] == "GANomaly" and
    arguments["generator"]["GANomaly"]["mask_generator_input_images_percent"] > 0.):
    # Image mask block pixel sizes list of lists.
    arguments["generator"]["GANomaly"]["image_mask_block_sizes"] = (
        image_masks.calculate_image_mask_block_sizes_per_resolution(
            num_resolutions=num_conv_blocks,
            min_height=arguments["generator"]["projection_dims"][0],
            min_width=arguments["generator"]["projection_dims"][1],
            pixel_mask_percent=(
                arguments["generator"]["GANomaly"][
                    "mask_generator_input_images_percent"]
            )
        )
    )

In [ ]:
arguments

In [ ]:
with open("proganomaly_json_config.json", "w") as outfile:
    json.dump(arguments, outfile)

In [ ]:
%%bash
gsutil cp proganomaly_json_config.json gs://.../proganomaly

These are the overrides of the JSON we just made. This way we don't have to keep recreating and copying the JSON to GCS and can make small, quick changes. Note however, that the nested dictionary structure has to match the JSON so that it correctly overrides paramters. Also note that since this is being sent via command line that this can not be extremely large. For that, you should modify the JSON.

In [ ]:
overrides = {
    "generator": {
        "add_uniform_noise_to_fake_images": False,
        "GANomaly": {
            "use_unet_skip_connections": [True] * 9 + [False] * 0,
            "mask_generator_input_images_percent": 0.2,
            "use_shuffle_image_masks": True,
#             "add_uniform_noise_to_z": True,
            "add_uniform_noise_to_z": False,

        },
        "losses": {
            "GANomaly": {
                "x_minus_G_of_x_l2_loss_weight": 0.0
            }
        }
    },
    "training": {
        "output_dir": "gs://{BCKT}/{ROOT}/{DT}_{DATA}_{HW}_{SZ}_{BS}_{FILES}_{LEN}_{EXPORT}_{MISC}".format(
            BCKT=BUCKET,
            ROOT="gan/proganomaly/trained_models",
            DT=datetime.datetime.now().strftime("%Y%m%d%H%M%S"),
            DATA="normal_skin",
            HW="2_V100_mirrored",
            SZ="1024x1024",
            BS="bs_32_16x4_4_2x2_1",
            FILES="full_data_multiple_files",
            LEN="full_length",
            EXPORT="export_current",
            MISC="gen_linear_one_epoch_each_three_training_phases_gen_input_masks_experiment_0"
        ),
        "error_distribution": {
            "train_batch_size": 8
        },
        "dynamic_threshold": {
            "train_batch_size": 8
        }
    },
    "export": {
        "export_all_growth_phases": False
    }
}

In [ ]:
overrides

In [ ]:
json_overrides = json.dumps(overrides).replace("\"", "\'").replace(" ", ";")
json_overrides

In [ ]:
os.environ["IMAGE_SIZE"] = str(1024)
os.environ["JSON_CONFIG_GCS_PATH"] = "gs://.../proganomaly/proganomaly_json_config.json"
os.environ["JSON_OVERRIDES"] = json_overrides

In [ ]:
# %%bash
# export PYTHONPATH=${PYTHONPATH}:${PWD}/proganomaly_modules/training_module
# python3 -m trainer.task \
#     --job-dir=./tmp \
#     --json_config_gcs_path=${JSON_CONFIG_GCS_PATH} \
#     --json_overrides=${JSON_OVERRIDES}

## Train ProGANomaly model

In [ ]:
%%writefile config.yaml
trainingInput:
  scaleTier: CUSTOM
  masterType: n1-highmem-16
  masterConfig:
    acceleratorConfig:
      count: 1
      type: NVIDIA_TESLA_T4

In [ ]:
%%writefile config.yaml
trainingInput:
  scaleTier: CUSTOM
  masterType: n1-highmem-16
  masterConfig:
    acceleratorConfig:
      count: 2
      type: NVIDIA_TESLA_V100

In [ ]:
%%writefile config.yaml
trainingInput:
  scaleTier: CUSTOM
  masterType: n1-highmem-32
  masterConfig:
    acceleratorConfig:
      count: 4
      type: NVIDIA_TESLA_V100

In [ ]:
%%writefile config.yaml
trainingInput:
  scaleTier: CUSTOM
  masterType: n1-highmem-96
  masterConfig:
    acceleratorConfig:
      count: 8
      type: NVIDIA_TESLA_V100

In [ ]:
%%bash
JOBNAME=proganomaly_${IMAGE_SIZE}x${IMAGE_SIZE}_$(date -u +%y%m%d_%H%M%S)
echo ${OUTPUT_DIR} ${REGION} ${JOBNAME}
gcloud ai-platform jobs submit training ${JOBNAME} \
    --region=${REGION} \
    --module-name=trainer.task \
    --package-path=$PWD/../proganomaly_modules/training_module/trainer \
    --job-dir=gs://${BUCKET} \
    --staging-bucket=gs://${BUCKET} \
    --config=config.yaml \
    --runtime-version=${TFVERSION} \
    --python-version=${PYTHON_VERSION} \
    -- \
    --job-dir=./tmp \
    --json_config_gcs_path=${JSON_CONFIG_GCS_PATH} \
    --json_overrides=${JSON_OVERRIDES}

## A100s

In [ ]:
%%writefile proganomaly_modules/training_module/Dockerfile
FROM gcr.io/deeplearning-platform-release/tf2-gpu.2-3
COPY trainer /proganomaly_modules/training_module/trainer
RUN apt update && \
    apt install --yes python3-pip && \
    pip3 install --upgrade --quiet tensorflow==2.3

ENV PYTHONPATH ${PYTHONPATH}:/proganomaly_modules/training_module
ENTRYPOINT ["python3", "proganomaly_modules/training_module/trainer/task.py"]

In [ ]:
%%writefile proganomaly_modules/training_module/push_docker.sh
export PROJECT_ID=$(gcloud config list project --format "value(core.project)")
export IMAGE_REPO_NAME=proganomaly_training_container
export IMAGE_URI=us.gcr.io/${PROJECT_ID}/${IMAGE_REPO_NAME}

echo "Building  $IMAGE_URI"
docker build -f Dockerfile -t ${IMAGE_URI} ./
echo "Pushing $IMAGE_URI"
docker push ${IMAGE_URI}

In [ ]:
%%bash
cd proganomaly_modules/training_module
bash push_docker.sh

In [ ]:
%%writefile config.yaml
trainingInput:
  scaleTier: CUSTOM
  masterType: a2-highgpu-2g
  masterConfig:
    acceleratorConfig:
      count: 2
      type: NVIDIA_TESLA_A100

In [ ]:
%%bash
JOBNAME=proganomaly_${IMAGE_SIZE}x${IMAGE_SIZE}_$(date -u +%y%m%d_%H%M%S)
echo ${OUTPUT_DIR} ${REGION} ${JOBNAME}
IMAGE=us.gcr.io/ml-mps-aif-agan-p-d7d4/proganomaly_training_container
gcloud ai-platform jobs submit training ${JOBNAME} \
    --region=${REGION} \
    --staging-bucket=gs://${BUCKET} \
    --master-image-uri=${IMAGE} \
    --config=config.yaml \
    -- \
